# Анализ конверсий и бинарных метрик: Критерий Хи-квадрат

## 📊 Условие задачи

Мы провели A/B тест двух версий посадочной страницы:

| Группа | Посетители | Конверсии | Конверсия |
|--------|------------|-----------|-----------|
| **A** (новая) | 1,000 | 230 | 23.0% |
| **B** (старая) | 1,100 | 210 | 19.1% |

### ❓ Исследовательский вопрос
### **Есть ли статистически значимая разница в конверсии между группой A и группой B?**

## 🎯 Формулировка гипотез

- **H₀ (Нулевая гипотеза)**: Конверсии в группах A и B одинаковы

- **H₁ (Альтернативная гипотеза)**: Конверсии в группах A и B различаются

**Уровень значимости**: α = 0.05

## 📈 Таблица сопряженности

| Группа | Конвертировались | Не конвертировались | Всего |
|--------|------------------|---------------------|-------|
| **A** | 230 | 770 | 1,000 |
| **B** | 210 | 890 | 1,100 |
| **Всего** | **440** | **1,660** | **2,100** |



In [1]:
import numpy as np
import scipy.stats as stats
from statsmodels.stats.proportion import proportions_ztest

# Исходные данные
# Группа A
visitors_a = 1000
conversions_a = 230

# Группа B
visitors_b = 1100
conversions_b = 210

# 1. Создаем таблицу сопряженности для критерия Хи-квадрат
contingency_table = np.array([
    [conversions_a, visitors_a - conversions_a],
    [conversions_b, visitors_b - conversions_b]
])

print("Таблица сопряженности (наблюдаемые частоты):")
print("          Конвертир. | Не конвертир.")
print(f"Группа A:    {conversions_a:4d}    |    {visitors_a - conversions_a:4d}")
print(f"Группа B:    {conversions_b:4d}    |    {visitors_b - conversions_b:4d}")
print()

# 2. Проводим тест Хи-квадрат
chi2_stat, p_value, dof, expected = stats.chi2_contingency(contingency_table, correction=False)

"""

expected — это ожидаемые частоты в каждой ячейке таблицы, ебы нулевая гипотеза H0 была верна (т.е. если бы конверсии в группах не отличались).

Как интерпретировать:

Фактическая конверсия A: 230

Ожидаемая конверсия A: 209.5

Разница: +20.5

Это означает, что в группе A конверсия выше, чем можно было бы ожидать, если бы группы были одинаковыми. Именно эту разницу и "ловит" критерий Хи-квадрат.

Формула расчета:
Ожидаемая_частота = (Сумма_по_строке × Сумма_по_столбцу) / Общая_сумма

"""


print("=== Критерий Хи-квадрат ===")
print(f"Хи-квадрат статистика: {chi2_stat:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"Степени свободы: {dof}")
print()

# 3. ВЫВОДИМ ОЖИДАЕМЫЕ ЧАСТОТЫ (expected)
print("Ожидаемые частоты (expected):")
print("          Конвертир. | Не конвертир.")
print(f"Группа A:    {expected[0,0]:7.1f}  |    {expected[0,1]:7.1f}")
print(f"Группа B:    {expected[1,0]:7.1f}  |    {expected[1,1]:7.1f}")
print()

# 4. Объясняем расчет ожидаемых частот
print("=== Как рассчитываются ожидаемые частоты? ===")
total_visitors = visitors_a + visitors_b
total_conversions = conversions_a + conversions_b
total_non_conversions = (visitors_a - conversions_a) + (visitors_b - conversions_b)

# Расчет вручную для наглядности
expected_conv_a = (total_conversions * visitors_a) / total_visitors
expected_non_conv_a = (total_non_conversions * visitors_a) / total_visitors
expected_conv_b = (total_conversions * visitors_b) / total_visitors
expected_non_conv_b = (total_non_conversions * visitors_b) / total_visitors

print("Расчет ожидаемой конверсии для Группы A:")
print(f"Всего конверсий: {total_conversions}, Всего пользователей: {total_visitors}")
print(f"Ожидаемая конверсия A = ({total_conversions} × {visitors_a}) / {total_visitors} = {expected_conv_a:.1f}")
print(f"Фактическая конверсия A: {conversions_a}, Ожидаемая: {expected_conv_a:.1f}")
print(f"Разница: {conversions_a - expected_conv_a:.1f}")
print()

# 5. Альтернатива: Z-тест для двух долей
count = np.array([conversions_a, conversions_b])
nobs = np.array([visitors_a, visitors_b])
z_stat, z_p_value = proportions_ztest(count, nobs, alternative='two-sided')

print("=== Z-тест для двух долей ===")
print(f"Z-статистика: {z_stat:.4f}")
print(f"p-value: {z_p_value:.4f}")
print()

# 6. Рассчитываем конверсии
conversion_a = conversions_a / visitors_a * 100
conversion_b = conversions_b / visitors_b * 100
expected_conversion_a = expected_conv_a / visitors_a * 100

print("=== Результаты ===")
print(f"Конверсия группы A: {conversion_a:.2f}% (факт) vs {expected_conversion_a:.2f}% (ожидаем)")
print(f"Конверсия группы B: {conversion_b:.2f}%")
print(f"Абсолютная разница: {conversion_a - conversion_b:.2f}%")
print()

# 7. Интерпретация результата
alpha = 0.05

print("=== Вывод ===")
if p_value < alpha:
    print(f"✅ p-value ({p_value:.4f}) < {alpha}. Отвергаем нулевую гипотезу.")
    print("Статистически значимая разница в конверсии ЕСТЬ.")
    if conversion_a > conversion_b:
        print("Группа A показала лучший результат.")
    else:
        print("Группа B показала лучший результат.")
else:
    print(f"❌ p-value ({p_value:.4f}) >= {alpha}. Не отвергаем нулевую гипотезу.")
    print("Статистически значимой разницы в конверсии НЕТ.")

Таблица сопряженности:
[[230 770]
 [210 890]]

=== Критерий Хи-квадрат ===
Хи-квадрат статистика: 4.8328
p-value: 0.0279
Степени свободы: 1

=== Z-тест для двух долей ===
Z-статистика: 2.1984
p-value: 0.0279

=== Результаты ===
Конверсия группы A: 23.00%
Конверсия группы B: 19.09%
Абсолютная разница: 3.91%

=== Вывод ===
✅ p-value (0.0279) < 0.05. Отвергаем нулевую гипотезу.
Статистически значимая разница в конверсии ЕСТЬ.
Группа A показала лучший результат.
